In [1]:
import os
from collections import namedtuple, defaultdict
import xml.etree.ElementTree as ET
from shapely.geometry import MultiPolygon, Point, Polygon
import numpy as np
from threading import local, get_ident
from time import time
import random
from tqdm import tqdm

from multiprocessing import Process, Queue
from multiprocessing.dummy import Pool


In [2]:
import sys
if 'D:\\ACDC_LUNG_HISTOPATHOLOGY\\ASAP 1.9\\bin' not in sys.path:
    sys.path.append('D:\\ACDC_LUNG_HISTOPATHOLOGY\\ASAP 1.9\\bin')
import multiresolutionimageinterface as mir
reader = mir.MultiResolutionImageReader()

In [3]:
_SlideInfo=namedtuple('_Slide', ['slide_path', 'mask_path', 'xml_tissue', 'xml_cancer'])
_PolygonRaw = namedtuple('_Polygon', ['slide_path', 'coord'])
_Polygon=namedtuple('_Polygon', ['slide_path', 'area', 'polygon'])
_Data=namedtuple('_Data', ['slide_path', 'coord', 'label'])

In [4]:
def get_slideinfo(folder_with_data):
    slides_path=folder_with_data+'slides/'
    masks_path=folder_with_data+'masks/'
    xmls_path=folder_with_data+'annotations/'
    for name in os.listdir(slides_path):
        yield _SlideInfo(slides_path+name, 
                     masks_path+name.split('.')[0]+'_M.tif', 
                     xmls_path+name.split('.')[0]+'_G.xml',
                     xmls_path+name.split('.')[0]+'.xml')

In [5]:
def get_coordinate(_list, cancer=True):
    for el in _list:
        if cancer:
            xml_file = el.xml_cancer
        else:
            xml_file=el.xml_tissue
        for ann in ET.parse(xml_file).getroot().find('Annotations'):
            yx=[(int(float(coord.attrib['Y'])), int(float(coord.attrib['X']))) for coord in ann.find('Coordinates').findall('Coordinate')]
            yield _PolygonRaw(el.slide_path, yx)

In [6]:
def clear_small_polygon(_list):
    for el in _list:
        p=Polygon(el.coord)
        area=p.area
        if area > 1024*1024:
            yield _Polygon(el.slide_path, area, p)

In [7]:
def get_dict(source_files, cancer=True):
    d=defaultdict(list)
    for path, area, poly in clear_small_polygon(get_coordinate(source_files, cancer=cancer)):
        d[path].append((area, poly))
    return d

In [8]:
def to_multi(_list):
    return MultiPolygon([[p.exterior.coords, []] for _, p in _list]).buffer(0)

In [9]:
def subtraction(cancer, tissue):
    for k in (set(cancer) & set(tissue)):
        gt = tissue[k]
        canc = cancer[k]
        mp = to_multi(canc)
        tmp = ((poly.buffer(5) - mp) for _, poly in gt)
        tissue[k] = [(p.area, p) for p in tmp]
    return (cancer, tissue)

In [10]:
_TLS = local()
def tls_prng():
    # pylint: disable=no-member
    try:
        return _TLS.prng
    except AttributeError:
        _TLS.prng = np.random.RandomState(
            (get_ident() + np.random.get_state()[1][0]) % 2**32
        )
        return _TLS.prng

In [11]:
def get_random_point(polygon, number):
    prng = tls_prng()

    if not polygon.area:
        return

    minx, miny, maxx, maxy = polygon.bounds
    for _ in range(number):
        while True:
            p = Point(prng.uniform(minx, maxx), prng.uniform(miny, maxy))
            if polygon.contains(p):
                yield (int(p.coords[0][0]), int(p.coords[0][1]))
                break

In [12]:
def get_square(point, size):
    return Polygon([(point[0]-size/2, point[1]-size/2), 
                    (point[0]-size/2, point[1]+size/2), 
                    (point[0]+size/2, point[1]+size/2), 
                    (point[0]+size/2, point[1]-size/2)])

In [ ]:
slideinfo=list(get_slideinfo('../data/'))
np.random.shuffle(slideinfo)
train_slideinfo, valid_slideinfo = (slideinfo[:-len(slideinfo) // 10], slideinfo[-len(slideinfo) // 10:])
# train_cancer=get_dict(train_slideinfo, cancer=True)
# train_tissue=get_dict(train_slideinfo, cancer=False)
# valid_cancer=get_dict(valid_slideinfo, cancer=True)
# valid_tissue=get_dict(valid_slideinfo, cancer=False)
# train_cancer, train_tissue=subtraction(train_cancer, train_tissue)
# valid_cancer, valid_tissue=subtraction(valid_cancer, valid_tissue)

In [ ]:
# nbpoints=1500
# i=0
# while i < nbpoints:
#     rnd_slide=random.choice(list(train_cancer))
#     rnd_polygon_info=random.choice(train_cancer[rnd_slide])
#     rnd_point=list(get_random_point(rnd_polygon_info[1], (256,256), 0, 1))[0]
#     square=get_square(rnd_point, 316)
#     if rnd_polygon_info[1].contains(square):
#         i+=1
#         print(rnd_slide, rnd_point)

In [ ]:
_Data=namedtuple('_Data', ['slide_path', 'coord', 'label'])

def get_data(_defdict, nbpoints, patch_size, zoom, cancer=True):
    np.random.seed(int(time() * 1000) % 2**32)
    reader = mir.MultiResolutionImageReader()
    for _ in tqdm(range(nbpoints)):
        while True:
            rnd_slide=random.choice(list(_defdict))
            rnd_polygon_info=random.choice(_defdict[rnd_slide])
            rnd_point=list(get_random_point(rnd_polygon_info[1], 1))[0]
            square=get_square(rnd_point, 316)
            if rnd_polygon_info[1].contains(square):
                mask = reader.open(rnd_slide.replace('slides', 'masks').replace('.tif', '_M.tif'))
                slide=reader.open(rnd_slide)
                slide_patch= slide.getUCharPatch(startY=rnd_point[0], startX=rnd_point[1], height=316, width=316, level=zoom)
                mask_patch = mask.getUCharPatch(startY=rnd_point[0], startX=rnd_point[1], height=316, width=316, level=zoom)
                mask_patch = mask_patch.astype(bool)
                values, counts = np.unique(mask_patch, return_counts=True)
                if cancer:
                    if sum(values * counts) / sum(counts) * 100==100.0 and len(np.unique(slide_patch))>225:
                        yield _Data(rnd_slide, rnd_point, 1)
                        break
                else:
                    if sum(values * counts) / sum(counts) * 100==0.0 and len(np.unique(slide_patch))>225:
                        yield _Data(rnd_slide, rnd_point, 0)
                        break

In [ ]:
train_tissue=list(get_data(train_tissue, 3000, (256,256), 0, False))

In [ ]:
train_cancer=list(get_data(train_cancer, 3000, (256,256), 0, True))

In [ ]:
train_tissue

In [ ]:
train=train_cancer+train_tissue

In [ ]:
len(train)

In [ ]:
random.shuffle(train)

In [13]:
def get_data(slideinfo, nbpoints, patch_size, zoom):
    np.random.seed(int(time() * 1000) % 2**32)
    reader = mir.MultiResolutionImageReader()
    cancer=get_dict(slideinfo, cancer=True)
    tissue=get_dict(slideinfo, cancer=False)
    cancer, tissue=subtraction(cancer, tissue)
    for _defdict in (cancer, tissue):
        for _ in tqdm(range(nbpoints)):
            while True:
                rnd_slide=random.choice(list(_defdict))
                rnd_polygon_info=random.choice(_defdict[rnd_slide])
                rnd_point=list(get_random_point(rnd_polygon_info[1], 1))[0]
                square=get_square(rnd_point, 316)
                if rnd_polygon_info[1].contains(square):
                    mask = reader.open(rnd_slide.replace('slides', 'masks').replace('.tif', '_M.tif'))
                    slide=reader.open(rnd_slide)
                    slide_patch= slide.getUCharPatch(startY=rnd_point[0], startX=rnd_point[1], height=316, width=316, level=zoom)
                    mask_patch = mask.getUCharPatch(startY=rnd_point[0], startX=rnd_point[1], height=316, width=316, level=zoom)
                    mask_patch = mask_patch.astype(bool)
                    values, counts = np.unique(mask_patch, return_counts=True)
                    if cancer and sum(values * counts) / sum(counts) * 100==100.0 and len(np.unique(slide_patch))>225:
                        yield _Data(rnd_slide, rnd_point, 1)
                        break
                    if tissue and sum(values * counts) / sum(counts) * 100==0.0 and len(np.unique(slide_patch))>225:
                        yield _Data(rnd_slide, rnd_point, 0)
                        break

In [14]:
slideinfo=list(get_slideinfo('../data/'))
np.random.shuffle(slideinfo)
train_slideinfo, valid_slideinfo = (slideinfo[:-len(slideinfo) // 10], slideinfo[-len(slideinfo) // 10:])
train=list(get_data(train_slideinfo, 100, (256,256), 0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 21.98it/s]


In [ ]:
import pickle

In [ ]:
with open('../data/train.pickle', 'wb') as f:
    pickle.dump(train, f)

In [15]:
valid=list(get_data(valid_slideinfo, 10, (256,256), 0))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 28.15it/s]


In [ ]:
with open('../data/valid.pickle', 'wb') as f:
    pickle.dump(valid, f)

In [16]:
from torch.utils.data import Dataset
# from ._mir_hook import mir
from PIL import Image
from torchvision import transforms as T

_READER = mir.MultiResolutionImageReader()


class TrainDataset(Dataset):
    def __init__(self, _list, patch_size, zoom, transform=None):
        self._list = _list
        self.patch_size = patch_size
        self.zoom = zoom
        self.transform = transform

    def __len__(self):
        return len(self._list)

    def __getitem__(self, index):
        slide = _READER.open(self._list[index].slide_path)
        X = slide.getUCharPatch(
            startY=self._list[index].coord[0],
            startX=self._list[index].coord[1],
            height=316,
            width=316,
            level=self.zoom
        )
        X = Image.fromarray(X).convert('RGB')
        if self.transform is not None:
            X = self.transform(X)
        else:
            X = T.Compose([T.ToTensor()])(X)
        Y = self._list[index].label

        return X, Y


In [17]:
t_dataset=TrainDataset(train, (256,256), 0)

In [18]:
patch_size=(256,256)
transforms = T.Compose(
    [
        T.RandomVerticalFlip(),
        T.RandomHorizontalFlip(),
        T.RandomRotation(90, resample=Image.BILINEAR),
        T.CenterCrop(patch_size),
        T.ToTensor()
    ]
)

In [19]:
train_gen=TrainDataset(train, (256,256), 0, transforms)

In [22]:
s1=list(train_gen)[0]